In [42]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import torch.utils.model_zoo as model_zoo

from easydict import EasyDict

import os
from glob import glob
import pickle
import random
import math

import cv2
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cv2.__version__

'4.5.4-dev'

In [3]:
base_path = "kfood"

In [4]:
food_list = os.listdir(base_path)

In [5]:
food_list

['train0.csv',
 'train1.csv',
 'train2.csv',
 '구이',
 '국',
 '기타',
 '김치',
 '나물',
 '떡',
 '만두',
 '면',
 '무침',
 '밥',
 '볶음',
 '쌈',
 '음청류',
 '장',
 '장아찌',
 '적',
 '전',
 '전골',
 '조림',
 '죽',
 '찌개',
 '찜',
 '탕',
 '튀김',
 '한과',
 '해물',
 '회']

In [6]:
def CV_READ(filePath, mode) : 
    stream = open( filePath.encode("utf-8") , "rb") 
    bytes = bytearray(stream.read()) 
    numpyArray = np.asarray(bytes, dtype=np.uint8) 
    # cv2.IMREAD_UNCHANGED
    return cv2.imdecode(numpyArray , mode)

In [9]:
small_to_large_dict = dict()
small_category_id_dict = dict()
for large_category in food_list[3:] :
  for s in os.listdir(os.path.join(base_path, large_category, large_category)) :
    i = glob.glob(os.path.join(base_path, large_category, large_category, s) + '/*.jpg')
    i2 = glob.glob(os.path.join(base_path, large_category, large_category, s) +'/*.png')
    if i :
        img = i[0]
        img = os.path.basename(img)
        id = img.split('_')[1]
    elif i2 :
        img = os.path.basename(img2)
        img = i2[0]
        id = img.split('_')[1]
    else :
        id = -1
    small_to_large_dict[s] = large_category
    small_category_id_dict[s] = id


small_category_path_dict = dict()
for idx, category_name in enumerate(small_category_id_dict) : 
  large_category = small_to_large_dict[category_name]
  small_category_path_dict[category_name] = os.path.join(base_path, large_category, large_category, category_name)

In [10]:
def resolution_key_with_channel(x, y, z) : 
  return str(str(x)+'X'+str(y)+'X'+str(z))
def resolution_key(x, y) : 
  return str(str(x)+'X'+str(y))

In [12]:
resolution_dict_with_channel = dict()
resolution_dict = dict()
small_category_count_dict = dict()
None_case_list = []
None_case = 0
smallest_X = 99999
smallest_Y = 99999
x_path = ''
y_path = ''
for small_category in small_category_path_dict : 
  img_list = glob.glob(small_category_path_dict[small_category] + '/*')
  for img in img_list :
    root, ext = os.path.splitext(img)
    if ext not in ['.jpg', '.png', 'jpeg'] :
      continue
    image = CV_READ(img, mode=cv2.IMREAD_COLOR)
    if image is None : 
      None_case += 1
      None_case_list.append(img)
      continue
    height, width, channel = image.shape
    #height, width = image.size
    if height <= smallest_X : 
        smallest_X = height
        x_path = img
    if width <= smallest_Y :
        smallest_Y = width
        y_path = img
    key = resolution_key(height, width)
    key_with_c = resolution_key_with_channel(height, width, channel)
    if key in resolution_dict : 
      resolution_dict[key] += 1
    else :
      resolution_dict[key] = 1

    if key_with_c in resolution_dict_with_channel : 
      resolution_dict_with_channel[key_with_c] += 1
    else :
      resolution_dict_with_channel[key_with_c] = 1

    if small_category in small_category_count_dict : 
      small_category_count_dict[small_category] += 1
    else : 
      small_category_count_dict[small_category] = 1

In [47]:
print(None_case)
print(None_case_list)

230
['kfood\\구이\\구이\\갈비구이\\Img_000_0145.jpg', 'kfood\\구이\\구이\\갈비구이\\Img_000_0197.jpg', 'kfood\\구이\\구이\\갈비구이\\Img_000_0215.jpg', 'kfood\\구이\\구이\\갈비구이\\Img_000_0253.jpg', 'kfood\\구이\\구이\\갈치구이\\Img_001_0067.jpg', 'kfood\\구이\\구이\\갈치구이\\Img_001_0286.jpg', 'kfood\\구이\\구이\\갈치구이\\Img_001_0355.jpg', 'kfood\\구이\\구이\\닭갈비\\Img_004_0483.jpg', 'kfood\\구이\\구이\\닭갈비\\Img_004_0533.jpg', 'kfood\\구이\\구이\\불고기\\Img_007_0578.jpg', 'kfood\\구이\\구이\\불고기\\Img_007_0597.jpg', 'kfood\\구이\\구이\\삼겹살\\Img_008_0433.jpg', 'kfood\\구이\\구이\\삼겹살\\Img_008_0460.jpg', 'kfood\\구이\\구이\\삼겹살\\Img_008_0488.jpg', 'kfood\\구이\\구이\\삼겹살\\Img_008_0563.jpg', 'kfood\\구이\\구이\\장어구이\\Img_009_0394.jpg', 'kfood\\구이\\구이\\장어구이\\Img_009_0437.jpg', 'kfood\\구이\\구이\\황태구이\\Img_012_0159.jpg', 'kfood\\구이\\구이\\황태구이\\Img_012_0266.jpg', 'kfood\\구이\\구이\\황태구이\\Img_012_0324.jpg', 'kfood\\구이\\구이\\훈제오리\\Img_013_0214.jpg', 'kfood\\국\\국\\떡국_만두국\\Img_015_0294.jpg', 'kfood\\국\\국\\떡국_만두국\\Img_015_0320.jpg', 'kfood\\국\\국\\떡국_만두국\\Img_015_0503.jpg', 'kfood\\국\\국\\미역국\\

In [13]:
# 우선 각 소분류 음식마다 모든 데이터 path가 저장된 dict을 만든다 (train용 하나, test용 하나)
# loader가 idx, (food_id, food_path_list) in data_dict() 하면 
# sample_data에서 전달받은 food_path_list에서 cv2로 이미지를 로드하고 (이미지데이터 리스트) 를 넘겨준다 (if shuffle=True 이면 shuffle)
# loader는 전달받은 리스트를 배치 갯수만큼 끊어서 target_id와 함께 yield 한다

train_dict = dict()
test_dict = dict()

for small_food in small_category_path_dict : 
    total_img = os.listdir(small_category_path_dict[small_food])
    list = []
    for food_img in total_img : 
        root, ext = os.path.splitext(food_img)
        if ext not in ['.jpg', '.png', 'jpeg'] :
            continue
        path = os.path.join(base_path, small_to_large_dict[small_food], small_to_large_dict[small_food], small_food, food_img)
        if path in None_case_list : 
            continue
        target = small_category_id_dict[small_food]
        list.append(path)
    np.random.shuffle(list)
    train_dict[target] = list[100:]
    test_dict[target] = list[:100]

print(test_dict[small_category_id_dict['추어탕']])
        

['kfood\\탕\\탕\\추어탕\\Img_139_0641.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0652.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0102.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0014.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0000.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0452.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0876.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0084.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0653.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0400.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0772.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0295.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0775.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0770.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0386.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0367.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0941.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0753.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0640.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0376.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0635.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0852.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0187.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0771.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0687.jpg', 'kfood\\탕\\탕\\추어탕\\Img_139_0915.jpg', 'kfood\\탕\\

In [14]:
def sample_data(data_path_list, shuffle=False) :
    total_num = len(data_path_list)
    img_idx = np.arange(total_num)
    if shuffle :
        np.random.shuffle(img_idx)
        
    list = []
    for idx in img_idx :
        path = data_path_list[idx]
        if path in None_case_list : 
            continue
        img = CV_READ(path, mode = cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize=(64, 64), interpolation=cv2.INTER_AREA)
        img = np.expand_dims(img, axis=0)
        list.append(img)
    
    list = np.array(list)
    return list

In [27]:
def data_loader(data_dict, batch_size, shuffle=False) : 
    total_img = []
    total_food_id = []
    for idx, (food_id, food_path_list) in enumerate(data_dict.items()) :
        img_data_list = sample_data(food_path_list, shuffle)
        for cur_img in img_data_list : 
            total_img.append((food_id, cur_img))
    
    total_num = len(total_img)
    total_idxs = np.arange(total_num)
    if shuffle:
        np.random.shuffle(total_idxs)
        
    for start_idx in range(0, total_num, batch_size) :
        end_idx = start_idx + batch_size
        index_list = total_idxs[start_idx:end_idx]
        
        id = []
        x = []
        
        for cur_idx in index_list : 
            food_id, food_img = total_img[cur_idx]
            id.append(int(food_id))
            x.append(food_img)
        
        yield id, x
    

In [36]:
# model
class CNN(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=6, stride=2, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=6, stride=2, padding=2)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 150)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.size(0), -1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [37]:
def id_to_onehot(id, total_range) : 
    onehots = []
    for i in id : 
        l = np.zeros((total_range))
        l[int(i)] = 1.0
        onehots.append(l)
    onehots = np.array(onehots)
    return torch.Tensor(onehots).long().cuda() 

In [38]:
class Trainer():
    def __init__(self, args):
        self.args = args
        
        self.model = CNN(args)
        if torch.cuda.is_available():
            self.model.cuda()
        self.loss = nn.CrossEntropyLoss()
        self.optim = torch.optim.Adam(self.model.parameters(), lr = 0.001)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
    def train(self):
        # train
        opt_epoch = 1
        min_val_loss = 1e9
        for epoch in range(self.args.epoch):
            train_loader = data_loader(train_dict, 10, shuffle=True)
            for id, x in train_loader :
                inputs = torch.tensor(x).to(device=self.device, dtype=torch.float)
                label = torch.tensor(id).to(device=self.device)
                
                pred = self.model(inputs)
                loss = self.loss(pred, label)
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
            
            if (epoch+1)%50 == 0 :
                val_loss = self.validate()
                print ('Validation Loss: {:.20f}'.format(val_loss))
                if val_loss < min_val_loss:
                    min_val_loss = val_loss
                    opt_epoch = epoch+1
                
            print(f'{epoch}: {loss}')
            torch.save(self.model.state_dict(), 'checkpoint' + str(epoch+1) + '.pt')
        print("Best Epoch : {}".format(opt_epoch))

    def validate(self) : 
        val_loader = data_loader(test_dict, 10, shuffle=True)
        self.model.eval()
        total_loss = 0
        total_step = 0
        with torch.no_grad() :
            for id, x in val_loader : 
                id = int(id)
                inputs = torch.tensor(x).to(device=self.device, dtype=torch.float)
                label = torch.tensor(id).to(device=self.device)

                pred = self.model(inputs)
                loss = self.loss(pred, label)
                total_loss += loss.item()
                total_step += 1
        return total_loss / total_step
                
    def test(self):
        # test
        self.model.load_state_dict(torch.load(self.args.ckpt))

In [39]:

from easydict import EasyDict
def get_args():
    args = EasyDict({
        "epoch":100,
        "batch_size":10,
        "mode":'train',
        "ckpt":1,
        "device":'cuda'
    })
    return args

In [ ]:
args = get_args()
trainer = Trainer(args)

if args.mode == 'train':
    trainer.train()
else:
    trainer.test()